In [1]:
import pandas as pd
#작은 브라우저로 웹사이트의 내용과 정보를 불러옴
import requests
# 웹사이트의 html 태크를 파싱하는데 사용
from bs4 import BeautifulSoup as bs
import random
import time
# 진행 상황 표시
from tqdm import tqdm, trange
# 정규표현식
import re
import konlpy
from konlpy.tag import Hannanum, Kkma, Okt # Mecab

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

#그래프 스타일 지정
plt.style.use('seaborn') 
# ‘default’,‘dark_background’,‘classic’,‘ggplot’,‘seaborn’

import matplotlib as mpl
mpl.rcParams.update({'font.size':25})

plt.rc('font',family='Malgun Gothic')
#plt.rc('font', family='AppleGothic')

%matplotlib inline

In [4]:
def get_suggestion_list(pnum):
    base_url = f'https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300102&cateCd=&searchText=&status1Cd=00002&Status2Cd=&sugMonthTp=&orderKey=registDtDesc'
    response = requests.get(base_url)

    if response.status_code == 200:
        html = bs(response.text, 'html.parser')
        tags = html.select('#content > div.suggestion_list > ul ')[0].find_all('a')
        #print(tags)
        if not tags:
            return(suggestion_list)
        else:
            for tag in tags:
                suggestion_list.append(tag['href'])
        pnum += 1
        get_suggestion_list(pnum)
    else:
        return(suggestion_list)

In [5]:
suggestion_list = []
pnum = 1

get_suggestion_list(pnum)
print(suggestion_list[:10])
print(len(suggestion_list))

['/ucms/ogp/sug/view.do?menuNo=300102&sgId=158&pnum=1', '/ucms/ogp/sug/view.do?menuNo=300102&sgId=157&pnum=1', '/ucms/ogp/sug/view.do?menuNo=300102&sgId=156&pnum=1', '/ucms/ogp/sug/view.do?menuNo=300102&sgId=155&pnum=1', '/ucms/ogp/sug/view.do?menuNo=300102&sgId=154&pnum=1', '/ucms/ogp/sug/view.do?menuNo=300102&sgId=153&pnum=2', '/ucms/ogp/sug/view.do?menuNo=300102&sgId=152&pnum=2', '/ucms/ogp/sug/view.do?menuNo=300102&sgId=151&pnum=2', '/ucms/ogp/sug/view.do?menuNo=300102&sgId=150&pnum=2', '/ucms/ogp/sug/view.do?menuNo=300102&sgId=149&pnum=2']
123


In [6]:
def get_suggestion_content(url):
    article = []
    base_url = base_url = f'https://www.innogov.go.kr{url}'
    response = requests.get(base_url)
    if response.status_code == 200:
        html = bs(response.text, 'html.parser')
        title = html.select('#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
        meta = html.select('#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
        category = meta[0].get_text(strip=True)
        start = meta[1].get_text(strip=True)
        end = meta[2].get_text(strip=True)
        author = meta[3].get_text(strip=True)
        vote = html.select('#counter')[0].get_text(strip=True)
        sgId = url.split('=')[2].split('&')[0]
        
        content = html.select('#content > div.vveiw_box1 > div.vveiw_cont > div > pre')[0].get_text(strip=True)
        
        article.append(sgId)
        article.append(title)
        article.append(category)
        article.append(content)
        article.append(start)
        article.append(end)
        article.append(vote)
        article.append(author)
        
        time.sleep(random.randint(1,2))
        return article

In [7]:
data = []
for i, suggestion in tqdm(enumerate(suggestion_list)):
    s = get_suggestion_content(suggestion)
    data.append(s)

123it [03:50,  1.92s/it]


In [12]:
col_names = ['sgId','title','category','content','start','end','vote','author']
col_names

['sgId', 'title', 'category', 'content', 'start', 'end', 'vote', 'author']

In [13]:
data = pd.DataFrame(data,columns=col_names)
data.head()

,sgId,title,category,content,start,end,vote,author
0,158,아이돌보미 서비스에 대한 건의 입니다.,보건복지,"육아 복지 서비스 중의 하나인 ""아이돌보미""서비스에 대한 의견입니다.아이돌보미 서비...",2019.03.19,2019.06.16,9,디딤돌
1,157,행정부와 사법부간 전자문서 유통시스템 구축,일반행정,"행정부(중앙, 지방)에서는 온나라 전자문서 시스템을 통해 공문을 주고 받을 수 있습...",2019.03.13,2019.06.10,6,한들바람
2,156,따뜻한 공동체 활성화를 위한 거점 공간 마련 필요.,공동체,○최근 동아리 및 모임시 함께 회의나 공동으로 모여서 이용할 수 있는 공간에 대한 ...,2019.03.08,2019.06.05,5,마이쭈샘
3,155,미세 먼지 저감 아이디어 - 자동차가 공기 청정기가 된다면?,환경,아래와 같이 정리했듯이 자동차를 공해의 주범이 아닌 공기 청정의 주역으로 바꾸는 아...,2019.03.07,2019.06.04,4,봉팔버미
4,154,담당자 제멋대로 운영되는 중소기업진흥공단(경기본부)의 정책자금 운영,국민참여,이번 정부 들어와서는 타 정부와는 다르게 규제혁신을 줄곧 추진하고있다하지만 실무에서...,2019.03.06,2019.06.03,4,포동이


In [14]:
data['category'].value_counts()

일반행정     22
기타       22
보건복지     17
안전       15
상생협력     10
인권/평등     9
국민참여      8
반부패청렴     6
환경        6
사회통합      5
공동체       3
Name: category, dtype: int64

In [15]:
data.to_csv('suggestion.csv',index=False,encoding='utf-8')
# data = pd.read_csv('suggestion.csv',encoding='utf-8')